In [1]:
import requests 
import pandas as pd 
import json 
import numpy as np
import pprint

from bs4 import BeautifulSoup

In [2]:
header = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36'}

In [3]:
api_key = 'ac80e2c89884220d09d71b0042b91aebc5f4cd1f'

In [4]:
url = 'https://www.gamespot.com/api/reviews/'

In [5]:
url_params = {
    'api_key': api_key,
    'format': 'json',
    'limit': 100,
            }


# dict_keys(['publish_date', 'update_date', 'review_type', 'id', 'authors', 'title', 'image', 'score', 'deck', 'good', 'bad', 'body', 'lede', 'game', 'releases', 'site_detail_url'])

In [6]:
def data_call(url_params):
    response = requests.get(url,headers = header, params = url_params)
    res_json = json.loads(response.text)
    
    return res_json['results']



text = data_call(url_params)
print(text[0])


{'publish_date': '1996-05-01 12:00:00', 'update_date': '1996-05-01 12:00:00', 'review_type': 'primary', 'id': 2531694, 'authors': 'Jeff Sengstack', 'title': 'Under a Killing Moon Review', 'image': {'square_tiny': 'https://gamespot1.cbsistatic.com/uploads/square_tiny/469/4693985/2305502-default-art--kubrick.jpg', 'screen_tiny': 'https://gamespot1.cbsistatic.com/uploads/screen_tiny/469/4693985/2305502-default-art--kubrick.jpg', 'square_small': 'https://gamespot1.cbsistatic.com/uploads/square_small/469/4693985/2305502-default-art--kubrick.jpg', 'original': 'https://gamespot1.cbsistatic.com/uploads/original/469/4693985/2305502-default-art--kubrick.jpg'}, 'score': '6.9', 'deck': 'Under a Killing Moon is an excellently wrapped package with some less than compelling contents.', 'good': '', 'bad': '', 'body': '<p>Under a Killing Moon (UAKM) is an excellently wrapped package with some less than compelling contents.</p>  <p>You play Tex Murphy, the bumbling, disheveled detective from two previou

In [7]:
columns = ['Title', 'Platform', 'Rating']
tester = pd.DataFrame(columns=columns)
tester.to_csv('gamespotdata.csv')

In [8]:
print(len(text))

100


In [10]:
results = []
def parse_data(lst):
    global results
    for data in lst:
        if 'releases' in data.keys():
            for release in data['releases']:
                if 'Xbox One' in release['platform']:
                    obj = {'title':data['title'], 'platforms': release['platform'], 'score': data['score']}
                    results.append(obj)
    
    return results       




In [11]:
results

[]

In [12]:
cur = 0

#set up a while loop to go through and grab the result 
while cur < 15049:
    url_params['offset'] = cur
    info = data_call(url_params)
    parse_data(info)
    
    cur += 100

In [33]:
df = pd.DataFrame(results)
df.head(10)

,title,platforms,score
0,Planescape: Torment Review,Xbox One,9.0
1,Icewind Dale Review,Xbox One,8.6
2,Baldur's Gate II: Shadows of Amn Review,Xbox One,9.2
3,Icewind Dale: Heart of Winter Review,Xbox One,5.7
4,Onimusha: Warlords Review,Xbox One,8.4
5,Baldur's Gate II: Throne of Bhaal Review,Xbox One,8.7
6,Neverwinter Nights Review,Xbox One,9.2
7,Neverwinter Nights Review,Xbox One,9.2
8,Industry Giant II Review,Xbox One,5.8
9,Neverwinter Nights: Shadows of Undrentide Review,Xbox One,7.8


In [34]:
titles = []
for title in df['title']:
    titles.append(title.replace(' Review',''))
        

In [35]:
df['Title'] = titles

In [36]:
df.drop(columns='title', inplace=True)

In [37]:
df.drop(columns='platforms',inplace=True)

In [38]:
df.head(5)

,score,Title
0,9.0,Planescape: Torment
1,8.6,Icewind Dale
2,9.2,Baldur's Gate II: Shadows of Amn
3,5.7,Icewind Dale: Heart of Winter
4,8.4,Onimusha: Warlords


In [21]:
df.to_csv('gamespot_reviews.csv')

In [26]:
df['Title'].unique()

array(['Planescape: Torment', 'Icewind Dale',
       "Baldur's Gate II: Shadows of Amn",
       'Icewind Dale: Heart of Winter', 'Onimusha: Warlords',
       "Baldur's Gate II: Throne of Bhaal", 'Neverwinter Nights',
       'Industry Giant II', 'Neverwinter Nights: Shadows of Undrentide',
       'XIII', 'Neverwinter Nights: Hordes of the Underdark',
       'Resident Evil 4', 'Titan Quest', 'Dead Rising', 'Yakuza', 'Okami',
       'Final Fantasy XII', 'Gears of War',
       'Super Monkey Ball: Banana Blitz', 'Titan Quest: Immortal Throne',
       'Resident Evil 4 Wii Edition', 'BioShock',
       "Bladestorm: The Hundred Years' War", 'Burnout Paradise',
       'Tales of Vesperia', 'Yakuza 2', 'Baja: Edge of Control',
       "Lock's Quest", 'de Blob', 'Burnout Paradise: The Ultimate Box',
       'Resident Evil 5', 'Red Faction: Guerrilla',
       'Ghostbusters The Video Game',
       'Red Faction: Guerrilla - Demons of the Badlands', 'Darksiders',
       'Resident Evil 5: Lost in Nightmar

In [39]:
drop_dup = df.drop_duplicates('Title')

In [40]:
drop_dup.head(20)

,score,Title
0,9.0,Planescape: Torment
1,8.6,Icewind Dale
2,9.2,Baldur's Gate II: Shadows of Amn
3,5.7,Icewind Dale: Heart of Winter
4,8.4,Onimusha: Warlords
5,8.7,Baldur's Gate II: Throne of Bhaal
6,9.2,Neverwinter Nights
8,5.8,Industry Giant II
9,7.8,Neverwinter Nights: Shadows of Undrentide
11,6.4,XIII


In [41]:
print(drop_dup.shape)

(732, 2)


In [43]:
drop_dup.to_csv('gamespot_reviews.csv')